In [1]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

In [2]:
client = dataiku.api_client()

In [3]:
project = client.get_project("DSPIPELINEFRAUD")

In [8]:
project_metadata = project.list_code_envs()

AttributeError: 'DSSProject' object has no attribute 'list_code_envs'

In [9]:
project_metadata

{'projectKey': 'DSPIPELINEFRAUD',
 'ownerLogin': 'user1',
 'projectType': 'REGULAR',
 'projectAppType': 'REGULAR',
 'permissionsVersion': 'LEGACY',
 'tutorialProject': False,
 'disableAutomaticTriggers': False,
 'commitMode': 'AUTO',
 'name': 'Credit Card Fraud predictions',
 'projectStatus': 'In use',
 'showInitials': False,
 'isProjectAdmin': True,
 'canReadProjectContent': True,
 'canWriteProjectContent': True,
 'canModerateDashboards': True,
 'canWriteDashboards': True,
 'canManageDashboardAuthorizations': True,
 'canManageExposedElements': True,
 'canManageAdditionalDashboardUsers': True,
 'canExportDatasetsData': True,
 'canReadDashboards': True,
 'canRunScenarios': True,
 'canExecuteApp': True,
 'canShareToWorkspaces': True,
 'ownerDisplayName': 'user1',
 'objectImgHash': 1647442710684,
 'isProjectImg': True,
 'defaultImgColor': '#a0a0a0',
 'imgPattern': 4,
 'contributors': [],
 'metrics': {'probes': [],
  'engineConfig': {'padRunsWithMetrics': False,
   'hive': {'active': True,